In [1]:
import pandas as pd
import itertools

import requests
from requests_html import HTML
from datetime import date
from tqdm.notebook import tqdm

def default_parameters(fmt, start_dt, end_dt):
    defaults = {
        # scheme and state
        0: {
            'PROGRAM': '%2Fstatistics%2Fpbs_csv',
            'REPIND': 'pbs_tab1',
            'RPT_FMT': '1',
            'cond': '+',
            'START_DT': start_dt,
            'END_DT': end_dt,
            'list': '',
            'fyear': '+',
            'year': '+',
            'by': '+',
            'statefmt': '%24statefw.',
            'vdim': 'newitem*%28scheme+all%29',
            'vdim2': '%28scheme%3D%27All+items%27+all%29',
            'hdim': '%28STATE+ALL%29*_replace_*SUM%3D%27+%27',
            'rts': '30',
            'title1': ''
        },
        # monthly scheme and state
        1: {
            'PROGRAM': '%2Fstatistics%2Fpbs_csv',
            'REPIND': 'pbs_tab1',
            'RPT_FMT': '2',
            'cond': '+',
            'START_DT': start_dt,
            'END_DT': end_dt,
            'list': '',
            'fyear': '+',
            'year': '+',
            'by': '+',
            'statefmt': '%24statefw.',
            'vdim': 'newitem*%28scheme+all%29',
            'vdim2': '*%28MOP+ALL%29+%28MOP%3D%27All+items%27+all%29',
            'hdim': '%28STATE+ALL%29*_replace_*SUM%3D%27+%27',
            'rts': '30',
            'title1': ''
        },
        # FY scheme and state
        2: {
            'PROGRAM': '%2Fstatistics%2Fpbs_csv',
            'REPIND': 'pbs_tab1',
            'RPT_FMT': '3',
            'cond': '+',
            'START_DT': start_dt,
            'END_DT': end_dt,
            'list': '',
            'fyear': 'fyear',
            'year': '+',
            'by': '+',
            'statefmt': '%24statefw.',
            'vdim': 'newitem*%28scheme+all%29',
            'vdim2': '*%28fyear+ALL%29+%28fyear%3D%27All+items%27+all%29',
            'hdim': '%28STATE+ALL%29*_replace_*SUM%3D%27+%27',
            'rts': '30',
            'title1': ''
        },
        # CY scheme and state
        3: {
            'PROGRAM': '%2Fstatistics%2Fpbs_csv',
            'REPIND': 'pbs_tab1',
            'RPT_FMT': '4',
            'cond': '+',
            'START_DT': start_dt,
            'END_DT': end_dt,
            'list': '',
            'fyear': '+',
            'year': 'year',
            'by': '+',
            'statefmt': '%24statefw.',
            'vdim': 'newitem*%28scheme+all%29',
            'vdim2': '*%28year+ALL%29+%28year%3D%27All+items%27+all%29',
            'hdim': '%28STATE+ALL%29*_replace_*SUM%3D%27+%27',
            'rts': '30',
            'title1': ''
        },
        # patient category
        4: {
            'PROGRAM': '%2Fstatistics%2Fpbs_csv',
            'REPIND': 'pbs_tab1',
            'RPT_FMT': '5',
            'cond': '+',
            'START_DT': start_dt,
            'END_DT': end_dt,
            'list': '',
            'fyear': '+',
            'year': '+',
            'by': '+',
            'statefmt': '%24statefw.',
            'vdim': 'newitem',
            'vdim2': 'all',
            'hdim': '%28scheme%3D%27%27*patcat%3D%27%27+ALL%29*_replace_*SUM%3D%27+%27',
            'rts': '30',
            'title1': ''
        },
        # month and patient category
        5: {
            'PROGRAM': '%2Fstatistics%2Fpbs_csv',
            'REPIND': 'pbs_tab1',
            'RPT_FMT': '6',
            'cond': '+',
            'START_DT': start_dt,
            'END_DT': end_dt,
            'list': '',
            'fyear': '+',
            'year': '+',
            'by': '+',
            'statefmt': '%24statefw.',
            'vdim': 'newitem',
            'vdim2': '*%28MOP+ALL%29+%28MOP%3D%27All+items%27+all%29',
            'hdim': '%28scheme%3D%27%27*patcat%3D%27%27+ALL%29*_replace_*SUM%3D%27+%27',
            'rts': '30',
            'title1': ''
        },
        # FY and patient category
        6: {
            'PROGRAM': '%2Fstatistics%2Fpbs_csv',
            'REPIND': 'pbs_tab1',
            'RPT_FMT': '7',
            'cond': '+',
            'START_DT': start_dt,
            'END_DT': end_dt,
            'list': '',
            'fyear': 'fyear',
            'year': '+',
            'by': '+',
            'statefmt': '%24statefw.',
            'vdim': 'newitem',
            'vdim2': '*%28fyear+ALL%29+%28fyear%3D%27All+items%27+all%29',
            'hdim': '%28scheme%3D%27%27*patcat%3D%27%27+ALL%29*_replace_*SUM%3D%27+%27',
            'rts': '30',
            'title1': ''
        },
        # CY and patient category
        7: {
            'PROGRAM': '%2Fstatistics%2Fpbs_csv',
            'REPIND': 'pbs_tab1',
            'RPT_FMT': '8',
            'cond': '+',
            'START_DT': start_dt,
            'END_DT': end_dt,
            'list': '',
            'fyear': '+',
            'year': 'year',
            'by': '+',
            'statefmt': '%24statefw.',
            'vdim': 'newitem',
            'vdim2': '*%28year+ALL%29+%28year%3D%27All+items%27+all%29',
            'hdim': '%28scheme%3D%27%27*patcat%3D%27%27+ALL%29*_replace_*SUM%3D%27+%27',
            'rts': '30',
            'title1': ''
        }
    }
    
    return defaults[fmt]

format_selection = {
    'Rows': {0: 'Scheme',
  1: 'Scheme + Month',
  2: 'Scheme + Financial year',
  3: 'Scheme + Calendar year',
  4: '',
  5: 'Month',
  6: 'Financial year',
  7: 'Calendar year'},
 'Columns': {0: 'State',
  1: 'State',
  2: 'State',
  3: 'State',
  4: 'Patient category',
  5: 'Patient category',
  6: 'Patient category',
  7: 'Patient category'}
}

def add_zeros(text):
    while len(text) < 6:
        text = '0' + str(text)
    return text

def grouper(chunk_size, item_codes):
    """
    Groups item codes into chunks of size n and returns a generator object
    Checks for instances of 'ITEM_CODE' and removes before returning
    """
    
    it = iter(item_codes)
    while True:
        chunk = tuple(itertools.islice(it, chunk_size))
        if any(code == 'ITEM_CODE' for code in chunk):
            chunk = list(chunk)
            chunk.remove('ITEM_CODE')
            chunk = tuple(chunk)
        if not chunk:
            return
        yield chunk
        
def generate_download_urls(fmt, start_dt, end_dt, item_codes, base_url):
    """
    Returns a generator of all the URLs needed to download
    """
    
    res = []
    
    for chunk in item_codes:        
        for report_type in ('SERVICES', 'BENEFIT'):
            params = default_parameters(fmt, start_dt, end_dt)
            # 'hdim' contains _replace_ which we replace with the report type required
            params['hdim'] = params['hdim'].replace('_replace_', report_type)
            params['list'] = '%27' + '%27%2C%27'.join(chunk) + '%27'
            
            url = base_url
            
            # concatenate parameters to form url
            for key, value in params.items():
                url += key + '=' + value + '&'
                
            # remove last & from url
            url = url[:-1]
            
            res.append((chunk, url, fmt))
            
    return res

def download_urls(urls, item_map):
    """
    Uses generator expression to download list of URLs.
    Returns list of item codes which failed to download (if any).
    """

    dl_fail_log = []
    df_list = []
    
    for chunk, url, fmt in tqdm(urls, desc='Downloading...'):
        if url.find('SERVICES') != -1:
            report_type = 'SERVICES'
        else:
            report_type = 'BENEFIT'
            
        df = get_data(url, report_type)
        if not isinstance(df, pd.DataFrame):
            dl_fail_log.extend(chunk)
            continue
        
        df_list.append(df)
    
    if dl_fail_log:
        print('Some downloads failed. See dl_fail_log.')
    
    print('Cleaning up...')
    df = pd.concat(df_list, sort=False)
    print('Done!')
    
    return df, dl_fail_log

def get_data(url, report_type):
    """
    Returns the downloaded data as a cleaned dataframe
    """
    
    data = pd.read_html(url) # returns as list
    if len(data) == 2:       # empty dataset
        return None
    
    df = data[1]
    return clean_df(df, report_type)
    

def clean_df(df, report_type):
    """
    Cleans the .xls item reports format and returns a df
    """
    
    df = df.droplevel(level=[0,2], axis=1)
    df.rename(columns={df.columns[0]:'Item Code', df.columns[1]:'Date'}, inplace=True)
    df.columns = [
        col.replace('Free', '').replace('SafetyNet', 'Safety Net')
        for col in df.columns
    ]

    df = df[(~df['Item Code'].isin(df['Date'].unique())) & (df['Date']!='Total')]

    df['Report_Type'] = report_type
    df['Date'] = pd.to_datetime(df['Date'])

    return df

def item_code_lookup(df, item_map):
    """
    Merges item report data with PBS item drug map data
    """
    
    df['ItemCodeLookup'] = df['Item Code'].map(lambda x: '0' * (6 -len(x)) + x)
    df = df.merge(item_map, 
                  left_on='ItemCodeLookup', 
                  right_on='ITEM_CODE')
    
    df.drop(['ItemCodeLookup','ITEM_CODE'], axis=1, inplace=True)
    
    df.rename(columns={'DRUG_NAME':'Item Name','FORM/STRENGTH':'Formulation','ATC5_Code':'ATC Code'},
              inplace=True)
    
    df['Item Name'] = df['Item Name'].str.title()
    
    return df

def get_item_map():
    """
    Downloads the PBS item drug map and saves into local dir. Returns this as a dataframe
    """
    
    item_map_url = 'https://www.pbs.gov.au/statistics/dos-and-dop/files/pbs-item-drug-map.csv'
    r = requests.get(item_map_url)
    with open('pbs-item-drug-map.csv', 'wb') as f:
        f.write(r.content)
    return pd.read_csv('pbs-item-drug-map.csv', encoding='latin-1')

def get_item_codes(item_map, n):
    """
    Returns the chunked (into size n) item codes from the PBS item drug map
    """
    
    return grouper(n, item_map.ITEM_CODE.unique())

def get_dates():
    """
    Gets the valid dates for item reports data
    """
    
    r = requests.get('http://medicarestatistics.humanservices.gov.au/statistics/pbs_item.jsp')

    html = HTML(html=r.content)

    dropdown = html.find('select[name=end_dt]')[0]
    return [tag.attrs['value'] for tag in dropdown.find('option')]

def run_program():
    """
    User prompts to run script.
    """
    
    dates = get_dates()
    
    print(f'Data available from {dates[0]} to {dates[-1]}\n')
    start_dt = input('Enter desired start date for downloads in format YYYYMM\n')
    end_dt = input('\nEnter desired start date for downloads in format YYYYMM\n')
    
    item_map = get_item_map()
    item_codes = get_item_codes(item_map, 400)

    base_url = 'http://medicarestatistics.humanservices.gov.au/statistics/do.jsp?_'
    
    urls = generate_download_urls(5, start_dt, end_dt, item_codes, base_url)
    
    df, dl_fail_log = download_urls(urls, item_map)
    
    while True:
        save_choice = input('Save downloaded dataset to local DIR? Y/N\n').lower()
        if save_choice in ('y', 'n'):
            break
        
    if save_choice == 'y':
        
        today = date.today()
        today = today.strftime('%Y-%m-%d')
        
        df.to_csv(f'{today}_{start_dt}_{end_dt}_Date_of_Processing_Utilisation.csv', index=False)
    
    return df, dl_fail_log

df, dl_fail_log = run_program()